In [1]:
import torchvision
from torchvision import transforms
import torch
from torch import nn
from torch.utils.data import DataLoader
import os

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [17]:
print(torch.cuda.is_available())

True


In [18]:
device = "cuda:0"

In [31]:
import torch

# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0.pth')
estado_fase_1 = torch.load('Fase_1.pth')


In [26]:
print(estado_fase_1["linear_relu_stack.4.weight"])
print(estado_fase_1["linear_relu_stack.4.bias"])

tensor([[ 0.0454, -0.0391, -0.0658,  ..., -0.0524, -0.0942,  0.0085],
        [-0.0193,  0.0436,  0.0238,  ...,  0.0301, -0.0064, -0.0019],
        [ 0.0270, -0.0110,  0.0054,  ..., -0.0842,  0.0650,  0.0347],
        ...,
        [-0.0154, -0.1233, -0.0039,  ...,  0.0591, -0.0296, -0.0098],
        [ 0.0290,  0.0072,  0.0139,  ...,  0.0007, -0.0433, -0.0186],
        [ 0.0401,  0.0267,  0.0197,  ...,  0.0993,  0.0004, -0.0284]],
       device='cuda:0')
tensor([-0.0329,  0.2524,  0.0166,  0.0098,  0.0994,  0.0797, -0.0611,  0.0681,
        -0.2061, -0.0385], device='cuda:0')


## Change f_1(5-10) to f_0(5-10)

In [32]:

# Transferir el clasificador de Fase_0 a Fase_1
estado_fase_1["linear_relu_stack.4.weight"][5:10, :] = estado_fase_0["linear_relu_stack.4.weight"][5:10, :]
estado_fase_1["linear_relu_stack.4.bias"][5:10] = estado_fase_0["linear_relu_stack.4.bias"][5:10]

In [37]:

model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_1)

<All keys matched successfully>

In [20]:

a, b = 0, 5
estado_fase_0["linear_relu_stack.4.weight"][a:b, :] = estado_fase_1["linear_relu_stack.4.weight"][a:b, :]
estado_fase_0["linear_relu_stack.4.bias"][a:b] = estado_fase_1["linear_relu_stack.4.bias"][a:b]

In [21]:
model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_0)

<All keys matched successfully>

## Change f_1(0-4) to f_(5-10)

In [5]:

# Transferir el clasificador de Fase_0 a Fase_1
a, b = 0, 5
estado_fase_1["linear_relu_stack.4.weight"][a:b, :] = estado_fase_1["linear_relu_stack.4.weight"][a:b, :]
estado_fase_1["linear_relu_stack.4.bias"][a:b] = estado_fase_1["linear_relu_stack.4.bias"][a:b]

In [27]:

model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_1)

<All keys matched successfully>

In [34]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    tarjet_prediction = []  # Lista para almacenar las etiquetas reales y predicciones

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # Guardar la etiqueta real y la predicción en la lista de tuplas
            tarjet_prediction.extend(list(zip(y.cpu().numpy(), pred.argmax(1).cpu().numpy())))

    test_loss /= num_batches
    correct /= size
    log_accuracy_loss.append((100*correct, test_loss))
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

    with open(f'logs/preditions_classifier_changed_f1_0_5_f0_5_10.txt', 'w') as archivo:
            # Escribe el valor de la variable en el archivo
            archivo.write(str(tarjet_prediction))
    print(f'El valor prediciones se ha guardado en el archivo.txt')
    

In [35]:
eval_data = torchvision.datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
eval_dataloader = DataLoader(eval_data, batch_size=10000, shuffle=True)
print("Se cargaron los datos correctamente")

Se cargaron los datos correctamente


In [38]:
learning_rate = 1e-3
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_classifier.parameters(), lr=learning_rate)
epochs = 1
log_accuracy_loss = []
test_loop(eval_dataloader, model_classifier, loss_fn)


Test Error: 
 Accuracy: 64.9%, Avg loss: 1.172056 

El valor prediciones se ha guardado en el archivo.txt
